In [1]:
from my_settings import apply_settings

apply_settings()

Applying os.environ variables:
Applying HUGGING_FACE_KEY
Applying HUGGINGFACEHUB_API_TOKEN
Applying OPENAI_API_KEY
Applying SERPAPI_API_KEY
Applying PINECONE_API_KEY
Applying PINECONE_API_ENV
Applying PINECONE_API_KEY_2
Applying PINECONE_API_ENV_2


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

In [2]:
# Initialize the OpenAI chat model
# llm = ChatOpenAI(model="gpt-4.1", temperature=0.7) 
from llm_factory import get_llm

llm = get_llm()

# Send a chat message
response = llm.invoke([
    HumanMessage(content="Tell me a fun fact about space.")
])

print(response.content)


NameError: name 'HumanMessage' is not defined

# Building Blocks: The Augmented LLM
https://langchain-ai.github.io/langgraph/tutorials/workflows/#set-up


In [5]:
#!pip install pydantic

In [6]:
# Schema for structured output
from pydantic import BaseModel, Field

class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Query that is optimized web search.")
    justification: str = Field(
        None, description="Why this query is relevant to the user's request."
    )


# Augment the LLM with schema for structured output
structured_llm = llm.with_structured_output(SearchQuery)

# # Invoke the augmented LLM
output = structured_llm.invoke("How does Calcium CT score relate to high cholesterol?")

# Define a tool
def multiply(a: int, b: int) -> int:
    return a * b

# Augment the LLM with tools
llm_with_tools = llm.bind_tools([multiply])

# Invoke the LLM with input that triggers the tool call
msg = llm_with_tools.invoke("What is 2 times 3?")

# Get the tool call
msg.tool_calls
#print(msg.content)

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'call_bNXBH1xJjowBT6TjSa2lv88k',
  'type': 'tool_call'}]

In [7]:
response = llm.invoke([
    HumanMessage(content="What is 5 times 6?")
])

print(response.content)


5 times 6 is **30**.


# Prompt chaining
https://langchain-ai.github.io/langgraph/tutorials/workflows/#prompt-chaining

In [8]:
#!pip install langgraph

In [9]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

In [10]:
# Graph state
class State(TypedDict):
    topic: str
    joke: str
    improved_joke: str
    final_joke: str


# Nodes
def generate_joke(state: State):
    """First LLM call to generate initial joke"""

    msg = llm.invoke(f"Write a short joke about {state['topic']}")
    response = {"joke": msg.content}
    print(f"generate_joke: {response}")
    print(State)
    print()
    return response

def check_punchline(state: State):
    """Gate function to check if the joke has a punchline"""

    # Simple check - does the joke contain "?" or "!"
    #if "?" in state["joke"] or "!" in state["joke"]:                
    if False:
        print('check_punchline: Pass')
        print(state)
        return "Pass"
    
    print(f"check_punchline: Fail")
    print(state)
    print()
    return "Fail"


def improve_joke(state: State):
    """Second LLM call to improve the joke"""

    msg = llm.invoke(f"Make this joke funnier by adding wordplay: {state['joke']}")
    response = {"improved_joke": msg.content}
    print(f"improve_joke: {response}")
    print()
    return response


def polish_joke(state: State):
    """Third LLM call for final polish"""

    msg = llm.invoke(f"Add a surprising twist to this joke: {state['improved_joke']}")
    
    response = {"final_joke": msg.content}
    print(f"polish_joke: {response}")
    print()
    return response


# Build workflow
workflow = StateGraph(State)

# Add nodes
workflow.add_node("generate_joke", generate_joke)
workflow.add_node("improve_joke", improve_joke)
workflow.add_node("polish_joke", polish_joke)

# Add edges to connect nodes
workflow.add_edge(START, "generate_joke")
workflow.add_conditional_edges(
    "generate_joke", check_punchline, {"Fail": "improve_joke", "Pass": END}
)
workflow.add_edge("improve_joke", "polish_joke")
workflow.add_edge("polish_joke", END)

# Compile
chain = workflow.compile()

# Show workflow
#display(Image(chain.get_graph().draw_mermaid_png()))

# Invoke
state = chain.invoke({"topic": "dogs"})
print("Initial joke:")
print(state["joke"])
print("\n--- --- ---\n")
if "improved_joke" in state:
    print("Improved joke:")
    print(state["improved_joke"])
    print("\n--- --- ---\n")

    print("Final joke:")
    print(state["final_joke"])
else:
    print("Joke failed quality gate - no punchline detected!")

generate_joke: {'joke': 'Why did the dog sit in the shade?\n\nBecause he didn’t want to be a hot dog!'}
<class '__main__.State'>

check_punchline: Fail
{'topic': 'dogs', 'joke': 'Why did the dog sit in the shade?\n\nBecause he didn’t want to be a hot dog!'}

improve_joke: {'improved_joke': "Certainly! Here's a punchier, wordplay-packed version:\n\nWhy did the dog sit in the shade?  \nBecause he didn’t want to become a *pup*-ular hot dog!"}

polish_joke: {'final_joke': 'Why did the dog sit in the shade?  \nBecause he didn’t want to become a *pup*-ular hot dog—  \nHe heard the last one got *relish*-ed!'}

Initial joke:
Why did the dog sit in the shade?

Because he didn’t want to be a hot dog!

--- --- ---

Improved joke:
Certainly! Here's a punchier, wordplay-packed version:

Why did the dog sit in the shade?  
Because he didn’t want to become a *pup*-ular hot dog!

--- --- ---

Final joke:
Why did the dog sit in the shade?  
Because he didn’t want to become a *pup*-ular hot dog—  
He he

# Weather Request

In [19]:
#!pip install requests beautifulsoup4 
from my_utils import get_weather

city = "Cape Town"
weather_forecast = get_weather(city=city)

msg = llm.invoke(f"""
You are a weather expert and looking at weather website content. 
I am going for a walk at Noon tomorrow, what clothes should I wear? And if it's at night how much light will there be?

# Instructions
* Be concise
* Rationalise your answer

# Answer
You should wear:
...

Reason:
...

# Weather Website Content
{weather_forecast}
""")

print("# LLM Response")
print(msg.content)
print()

print("# Weather Forecast Ground Truth")
print(weather_forecast)

# LLM Response
You should wear:  
A light jacket or sweater, long pants, and comfortable shoes. Bring a small umbrella or raincoat just in case.

Reason:  
At +13°C, it's cool but not cold—layers are comfortable. There's a chance of light rain (0.3mm), so light rain gear is sensible. The wind is moderate (28km/h), so a windbreaker helps.

At night, the waxing gibbous moon (🌔) means there will be moderate moonlight, making it brighter than usual but not as bright as a full moon.

# Weather Forecast Ground Truth
'Temperature (Actual):+13°C , Moon Phase:🌔, Rain:0.3mm, Wind:→28km/h'


# Recipes

In [23]:
import requests
from bs4 import BeautifulSoup
import json

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}


url = "https://www.allrecipes.com/recipe/245372/pad-thai-quinoa-bowl/"

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

#print(response.text)

# Find the script tag by id
script_tag = soup.find("script", id="allrecipes-schema_1-0")

# Parse and print JSON
if script_tag:
    data = json.loads(script_tag.string)
    print(json.dumps(data, indent=2))
else:
    print("Script tag not found.")


[
  {
    "@context": "http://schema.org",
    "@type": [
      "Recipe",
      "NewsArticle"
    ],
    "headline": "Pad Thai Quinoa Bowl",
    "datePublished": "2019-09-06T18:50:30.000-04:00",
    "dateModified": "2025-05-06T14:40:39.782-04:00",
    "author": [
      {
        "@type": "Person",
        "name": "Lisa"
      }
    ],
    "description": "This pad Thai quinoa bowl recipe includes nutty quinoa, chicken, and veggies in a zesty peanut sauce. Quinoa is gluten free, high in protein, and fiber.",
    "image": {
      "@type": "ImageObject",
      "url": "https://www.allrecipes.com/thmb/FZbamw4lrOsOR47PxOn0pkbmQhA=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/3162185-ac1b386618e84ebfaf76968fc79fe8b1.jpg",
      "height": 960,
      "width": 960
    },
    "video": {
      "@type": "VideoObject",
      "contentUrl": "https://content.jwplatform.com/videos/GHVRqj16-K3AjnAEN.mp4",
      "description": "This quinoa bowl is healthy and gluten free, and has an excellent 